<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/version2/preprocessing/CA_MergecombinedDatasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal

This colab merges for NY
* COVID-19 and population datasets
* Social distancing data
* Mask rule active data
* Google mobility data 

# Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
import warnings; warnings.simplefilter('ignore')

In [ ]:
drive.flush_and_unmount(600000)

In [ ]:
!ls '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021'

# Import package

In [4]:
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler as ss
import itertools
from scipy import linalg
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
import numpy as np
import matplotlib.pyplot as plt

# Load Data 

In [5]:
CA_COVID19_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/CA_CovidDataJan10.csv',low_memory=False)
CA_SD_data =pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/CA_SocialDistancingDataJan10.csv', low_memory=False)
mask_rule_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/Mask_Rule_Feb05.csv', low_memory=False)
CA_google_mobility_data = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/CA_GoogleMobilityDataJan10.csv', low_memory=False)




In [6]:
print(CA_COVID19_data.shape)
print(CA_SD_data.shape)
print(mask_rule_data.shape)
print(CA_google_mobility_data.shape)

(20474, 10)
(16534, 9)
(52, 7)
(18067, 14)


In [7]:
mask_rule_data.head(2)

,state_name,Law Date,Masks Required?,Type of Requirement,mask_required_rule,requirement_code,mask_severity
0,Mississippi,2020-08-04,Parts of State,NaN,6,-1,0.0
1,Vermont,2020-08-01,Entire State,Everywhere in public where social distancing i...,0,5,10.0


In [8]:
CA_mask_rule = mask_rule_data[mask_rule_data['state_name'] == 'California'].reset_index()
CA_mask_rule

,index,state_name,Law Date,Masks Required?,Type of Requirement,mask_required_rule,requirement_code,mask_severity
0,18,California,2020-06-18,Entire State,Everywhere in public where social distancing i...,0,5,10.0


In [9]:
CA_COVID19_data.head()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population
0,2020-01-22,California,6001.0,0,0.0,0,0,Alameda County,CA,1671329
1,2020-01-23,California,6001.0,0,0.0,0,0,Alameda County,CA,1671329
2,2020-01-24,California,6001.0,0,0.0,0,0,Alameda County,CA,1671329
3,2020-01-25,California,6001.0,0,0.0,0,0,Alameda County,CA,1671329
4,2020-01-26,California,6001.0,0,0.0,0,0,Alameda County,CA,1671329


In [10]:
CA_COVID19_data['FIPS'] = CA_COVID19_data['FIPS'].apply(lambda x :int(x))
CA_COVID19_data.head()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population
0,2020-01-22,California,6001,0,0.0,0,0,Alameda County,CA,1671329
1,2020-01-23,California,6001,0,0.0,0,0,Alameda County,CA,1671329
2,2020-01-24,California,6001,0,0.0,0,0,Alameda County,CA,1671329
3,2020-01-25,California,6001,0,0.0,0,0,Alameda County,CA,1671329
4,2020-01-26,California,6001,0,0.0,0,0,Alameda County,CA,1671329


# Combining COVID-19, population  and mask data

In [11]:
mask_value = CA_mask_rule['mask_severity'][0]
mask_value

10.0

In [ ]:
CA_mask_rule['Law Date'].values[0]

'2020-06-18'

In [12]:
CA_COVID19_data['mask_rule_severity'] = CA_COVID19_data['Date'].apply(lambda x : mask_value if x >= CA_mask_rule['Law Date'].values[0] else 0.0 )
CA_COVID19_data.head()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population,mask_rule_severity
0,2020-01-22,California,6001,0,0.0,0,0,Alameda County,CA,1671329,0.0
1,2020-01-23,California,6001,0,0.0,0,0,Alameda County,CA,1671329,0.0
2,2020-01-24,California,6001,0,0.0,0,0,Alameda County,CA,1671329,0.0
3,2020-01-25,California,6001,0,0.0,0,0,Alameda County,CA,1671329,0.0
4,2020-01-26,California,6001,0,0.0,0,0,Alameda County,CA,1671329,0.0


In [13]:
CA_COVID19_data[CA_COVID19_data['Date'] > '2020-06-18']

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population,mask_rule_severity
149,2020-06-19,California,6001,4702,119.0,64,2,Alameda County,CA,1671329,10.0
150,2020-06-20,California,6001,4924,119.0,222,0,Alameda County,CA,1671329,10.0
151,2020-06-21,California,6001,4924,119.0,0,0,Alameda County,CA,1671329,10.0
152,2020-06-22,California,6001,5007,120.0,83,1,Alameda County,CA,1671329,10.0
153,2020-06-23,California,6001,5267,121.0,260,1,Alameda County,CA,1671329,10.0
...,...,...,...,...,...,...,...,...,...,...,...
20469,2021-01-04,California,6115,3966,0.0,36,0,Yuba County,CA,78668,10.0
20470,2021-01-05,California,6115,4068,0.0,102,0,Yuba County,CA,78668,10.0
20471,2021-01-06,California,6115,4221,0.0,153,0,Yuba County,CA,78668,10.0
20472,2021-01-07,California,6115,4383,0.0,162,0,Yuba County,CA,78668,10.0


In [14]:
CA_COVID19_data.shape

(20474, 11)

# Combining COVID-19, population, mask data and Social distancing data

In [15]:
CA_SD_data.tail(2)

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
16532,2021-01-06,US,2,California,Yuba County,6115.0,1151,1.761,18
16533,2021-01-07,US,2,California,Yuba County,6115.0,915,0.082,0


In [16]:
CA_SD_data['fips'] = CA_SD_data['fips'].apply(lambda x :int(x))
CA_SD_data.head()

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
0,2020-03-01,US,1,California,NaN,6,451282,4.122,60
1,2020-03-02,US,1,California,NaN,6,457175,6.725,98
2,2020-03-03,US,1,California,NaN,6,472598,6.842,100
3,2020-03-04,US,1,California,NaN,6,481255,6.827,99
4,2020-03-05,US,1,California,NaN,6,458159,7.270,106


In [17]:
CA_SD_data.tail(2)

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
16532,2021-01-06,US,2,California,Yuba County,6115,1151,1.761,18
16533,2021-01-07,US,2,California,Yuba County,6115,915,0.082,0


In [18]:
CA_socialDistancing_combined = pd.merge(CA_COVID19_data,CA_SD_data,how='inner',left_on=['Date','County Name'],right_on=['date','admin2'])
CA_socialDistancing_combined.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population,mask_rule_severity,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
0,2020-03-01,California,6001,0,0.0,0,0,Alameda County,CA,1671329,0.0,2020-03-01,US,2,California,Alameda County,6001,16737,3.684,58
1,2020-03-02,California,6001,0,0.0,0,0,Alameda County,CA,1671329,0.0,2020-03-02,US,2,California,Alameda County,6001,17269,6.182,98


In [19]:
CA_socialDistancing_combined.shape

(16232, 20)

# Combining COVID-19, population, mask data, Social distancing data and Google Mobility data

In [20]:
CA_google_mobility_data.tail(2)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
18065,US,United States,California,Yuba County,NaN,NaN,6115.0,2021-01-04,-18.0,-8.0,-2.0,-32.0,-35.0,12.0
18066,US,United States,California,Yuba County,NaN,NaN,6115.0,2021-01-05,-14.0,-6.0,NaN,-26.0,-30.0,10.0


In [21]:
CA_all_data_merged = pd.merge(CA_socialDistancing_combined,CA_google_mobility_data,how='inner',left_on=['Date','County Name'],right_on=['date','sub_region_2'])
CA_all_data_merged.head(2)

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population,mask_rule_severity,date_x,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date_y,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,2020-03-01,California,6001,0,0.0,0,0,Alameda County,CA,1671329,0.0,2020-03-01,US,2,California,Alameda County,6001,16737,3.684,58,US,United States,California,Alameda County,NaN,NaN,6001.0,2020-03-01,6.0,9.0,15.0,1.0,4.0,0.0
1,2020-03-02,California,6001,0,0.0,0,0,Alameda County,CA,1671329,0.0,2020-03-02,US,2,California,Alameda County,6001,17269,6.182,98,US,United States,California,Alameda County,NaN,NaN,6001.0,2020-03-02,4.0,7.0,7.0,-1.0,3.0,0.0


In [22]:
PopDensity = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/population-density/Average_Household_Size_and_Population_Density_County.csv', low_memory=False)

In [23]:
CA_PopDensity = PopDensity[PopDensity['State'] == 'California']
CA_PopDensity.head()

,FID,COUNTYNS,GEOID,ALAND,AWATER,NAME,State,B25010_001E,B25010_001M,B25010_002E,B25010_002M,B25010_003E,B25010_003M,B01001_001E,B01001_001M,B01001_calc_PopDensity,created_user,created_date,last_edited_user,last_edited_date,B01001_calc_PopDensityM,SHAPE_Length,SHAPE_Area
186,187,1675839,6001,1909598013,216923745,Alameda County,California,2.81,0.01,2.93,0.02,2.68,0.02,1643700,NaN,860.757075,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.000000,2.677466,0.199990
187,188,1675840,6003,1912292630,12557304,Alpine County,California,3.71,1.02,3.66,0.97,4.00,2.65,1146,161.0,0.599281,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.084192,2.174214,0.199044
188,189,1675841,6005,1539933577,29470567,Amador County,California,2.42,0.07,2.43,0.08,2.40,0.19,37829,NaN,24.565345,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.000000,2.909839,0.161954
189,190,1675842,6007,4238438186,105311003,Butte County,California,2.55,0.03,2.48,0.04,2.64,0.05,227075,NaN,53.575159,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.000000,4.205815,0.455942
190,191,1675885,6009,2641829200,43797659,Calaveras County,California,2.57,0.07,2.49,0.08,2.83,0.22,45235,NaN,17.122606,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.000000,2.961851,0.276235


In [24]:
CA_PopDensity[CA_PopDensity['NAME'] == 'Yuba County']

,FID,COUNTYNS,GEOID,ALAND,AWATER,NAME,State,B25010_001E,B25010_001M,B25010_002E,B25010_002M,B25010_003E,B25010_003M,B01001_001E,B01001_001M,B01001_calc_PopDensity,created_user,created_date,last_edited_user,last_edited_date,B01001_calc_PopDensityM,SHAPE_Length,SHAPE_Area
243,244,277322,6115,1636913845,31057995,Yuba County,California,2.83,0.04,2.88,0.07,2.75,0.11,75493,NaN,46.119104,esri_demographics,2020/04/01 20:53:36,esri_demographics,2020/04/01 20:53:36,0.0,2.832244,0.174096


In [25]:
CA_all_data_merged['PopDensity'] = 0.00
for index, row in CA_all_data_merged.iterrows():
  county_fips = CA_all_data_merged['FIPS'][index]
  temp = CA_PopDensity[CA_PopDensity.eq(county_fips).any(1)].reset_index()
  CA_all_data_merged['PopDensity'][index] = temp['B01001_calc_PopDensity'][0]

In [27]:
CA_all_data_merged.head()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population,mask_rule_severity,date_x,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date_y,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity
0,2020-03-01,California,6001,0,0.0,0,0,Alameda County,CA,1671329,0.0,2020-03-01,US,2,California,Alameda County,6001,16737,3.684,58,US,United States,California,Alameda County,NaN,NaN,6001.0,2020-03-01,6.0,9.0,15.0,1.0,4.0,0.0,860.757075
1,2020-03-02,California,6001,0,0.0,0,0,Alameda County,CA,1671329,0.0,2020-03-02,US,2,California,Alameda County,6001,17269,6.182,98,US,United States,California,Alameda County,NaN,NaN,6001.0,2020-03-02,4.0,7.0,7.0,-1.0,3.0,0.0,860.757075
2,2020-03-03,California,6001,1,0.0,1,0,Alameda County,CA,1671329,0.0,2020-03-03,US,2,California,Alameda County,6001,18181,6.271,100,US,United States,California,Alameda County,NaN,NaN,6001.0,2020-03-03,5.0,12.0,41.0,0.0,1.0,-1.0,860.757075
3,2020-03-04,California,6001,1,0.0,0,0,Alameda County,CA,1671329,0.0,2020-03-04,US,2,California,Alameda County,6001,18495,6.209,99,US,United States,California,Alameda County,NaN,NaN,6001.0,2020-03-04,1.0,6.0,23.0,-3.0,1.0,0.0,860.757075
4,2020-03-05,California,6001,1,0.0,0,0,Alameda County,CA,1671329,0.0,2020-03-05,US,2,California,Alameda County,6001,17496,6.654,106,US,United States,California,Alameda County,NaN,NaN,6001.0,2020-03-05,1.0,6.0,12.0,-5.0,-1.0,1.0,860.757075


In [28]:
CA_all_data_merged.tail()

,Date,Province_State,FIPS,Confirmed,Deaths,New cases,New deaths,County Name,State,population,mask_rule_severity,date_x,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date_y,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity
15816,2021-01-01,California,6115,3898,0.0,40,0,Yuba County,CA,78668,10.0,2021-01-01,US,2,California,Yuba County,6115,1095,0.427,4,US,United States,California,Yuba County,NaN,NaN,6115.0,2021-01-01,-36.0,-23.0,18.0,-55.0,-68.0,22.0,46.119104
15817,2021-01-02,California,6115,3898,0.0,0,0,Yuba County,CA,78668,10.0,2021-01-02,US,2,California,Yuba County,6115,1007,1.502,15,US,United States,California,Yuba County,NaN,NaN,6115.0,2021-01-02,-17.0,-11.0,8.0,-41.0,-19.0,8.0,46.119104
15818,2021-01-03,California,6115,3930,0.0,32,0,Yuba County,CA,78668,10.0,2021-01-03,US,2,California,Yuba County,6115,1047,0.161,1,US,United States,California,Yuba County,NaN,NaN,6115.0,2021-01-03,-11.0,-7.0,33.0,NaN,-10.0,5.0,46.119104
15819,2021-01-04,California,6115,3966,0.0,36,0,Yuba County,CA,78668,10.0,2021-01-04,US,2,California,Yuba County,6115,822,0.074,0,US,United States,California,Yuba County,NaN,NaN,6115.0,2021-01-04,-18.0,-8.0,-2.0,-32.0,-35.0,12.0,46.119104
15820,2021-01-05,California,6115,4068,0.0,102,0,Yuba County,CA,78668,10.0,2021-01-05,US,2,California,Yuba County,6115,839,0.574,6,US,United States,California,Yuba County,NaN,NaN,6115.0,2021-01-05,-14.0,-6.0,NaN,-26.0,-30.0,10.0,46.119104


In [29]:
CA_all_data_merged.shape

(15821, 35)

In [30]:
CA_all_data_merged.columns

Index(['Date', 'Province_State', 'FIPS', 'Confirmed', 'Deaths', 'New cases',
       'New deaths', 'County Name', 'State', 'population',
       'mask_rule_severity', 'date_x', 'country_code', 'admin_level', 'admin1',
       'admin2', 'fips', 'samples', 'm50', 'm50_index', 'country_region_code',
       'country_region', 'sub_region_1', 'sub_region_2', 'metro_area',
       'iso_3166_2_code', 'census_fips_code', 'date_y',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity'],
      dtype='object')

In [31]:
required_cols = ['Date', 'Province_State', 'FIPS',  'New cases',
        'County Name', 'population', 'mask_rule_severity',
        'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity']

In [32]:
CA_all_data_selected = CA_all_data_merged[required_cols]
CA_all_data_selected.head(2)

,Date,Province_State,FIPS,New cases,County Name,population,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity
0,2020-03-01,California,6001,0,Alameda County,1671329,0.0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,860.757075
1,2020-03-02,California,6001,0,Alameda County,1671329,0.0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,860.757075


In [33]:
CA_all_data_selected.fillna(0)

,Date,Province_State,FIPS,New cases,County Name,population,mask_rule_severity,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity
0,2020-03-01,California,6001,0,Alameda County,1671329,0.0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,860.757075
1,2020-03-02,California,6001,0,Alameda County,1671329,0.0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,860.757075
2,2020-03-03,California,6001,1,Alameda County,1671329,0.0,6.271,100,5.0,12.0,41.0,0.0,1.0,-1.0,860.757075
3,2020-03-04,California,6001,0,Alameda County,1671329,0.0,6.209,99,1.0,6.0,23.0,-3.0,1.0,0.0,860.757075
4,2020-03-05,California,6001,0,Alameda County,1671329,0.0,6.654,106,1.0,6.0,12.0,-5.0,-1.0,1.0,860.757075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15816,2021-01-01,California,6115,40,Yuba County,78668,10.0,0.427,4,-36.0,-23.0,18.0,-55.0,-68.0,22.0,46.119104
15817,2021-01-02,California,6115,0,Yuba County,78668,10.0,1.502,15,-17.0,-11.0,8.0,-41.0,-19.0,8.0,46.119104
15818,2021-01-03,California,6115,32,Yuba County,78668,10.0,0.161,1,-11.0,-7.0,33.0,0.0,-10.0,5.0,46.119104
15819,2021-01-04,California,6115,36,Yuba County,78668,10.0,0.074,0,-18.0,-8.0,-2.0,-32.0,-35.0,12.0,46.119104


# Store file to Google Drive

In [ ]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'CA_alldatasets_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
CA_all_data_selected.to_csv(location+filename,index=False)

CA_alldatasets_Feb05.csv


# Combining Maryland and combined datasets

##  Load Preprocessed Social Distancing Inertia Data [Maryland University]

In [35]:
!ls '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'

 CA_alldatasets_Feb05.csv
 CA_alldatasets_Jan13.csv
 CA_CombinedData_Labeled_CDCGuideline_Feb05.csv
 CA_CombinedData_Labeled_CDCGuideline_Feb06.csv
 CA_CombinedData_Labeled_CDCGuideline_Jan15.csv
 CA_combined_Feb04.csv
 CA_CovidDataJan10.csv
 CA_GoogleMobilityDataJan10.csv
 CA_MarylandData_Labeled_CDCGuideline_Feb05.csv
 CA_MarylandData_Labeled_CDCGuideline_Feb06.csv
 CA_MarylandData_Labeled_CDCGuideline_Jan15.csv
 CA_processedMarylandData_Feb04.csv
 CA_processedMarylandData_Feb05.csv
 CA_processedMarylandData_Jan11.csv
 CA_SocialDistancingDataJan10.csv
 CA_socialDistancingInertiaData_Feb04.csv
 computed_CA_mobility_data.csv
 data-mask_for_all.csv
 election_results_2021.csv
 maryland_inertia_data_allstates_Feb04.csv
 maryland_inertia_data_allstates_Feb05.csv
 maryland_inertia_data_allstates_Jan11.csv
 maryland_inertia_raw_dec28_2020.csv
 Mask_Rule_Feb05.csv
 Mask_Rule_Jan10.csv
 NY_alldatasets_Jan13.csv
 NY_CombinedData_Labeled_CDCGuideline_Feb07.csv
 NY_CombinedData_Labeled_CDCGuideli

In [36]:
maryland_sd_inertia = pd.read_csv('/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/maryland_inertia_data_allstates_Feb05.csv',low_memory=False)
maryland_sd_inertia.head()

,index,State_Name,PopDensity,countyFIPS,County Name,State,population,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,date,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Population.density,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,mask_rule_severity
0,67518,California,860.757075,6001,Alameda County,CA,1671329,6001,Alameda County,6,50,32,2.41,26.1,0.4,38.0,0.20,2.21,0,1666753,-21.7,2020-01-01,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,0.0,0,0.9,3.9,5.7,0.3,0.0,0.0
1,67519,California,860.757075,6001,Alameda County,CA,1671329,6001,Alameda County,6,24,22,3.17,27.9,0.4,38.5,0.50,2.66,0,1666753,-5.8,2020-01-02,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,0.0,0,0.9,3.9,5.7,0.3,0.0,0.0
2,67520,California,860.757075,6001,Alameda County,CA,1671329,6001,Alameda County,6,21,21,3.28,27.2,0.4,41.0,0.51,2.78,0,1666753,-1.5,2020-01-03,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,0.0,0,0.9,3.9,5.7,0.3,0.0,0.0
3,67521,California,860.757075,6001,Alameda County,CA,1671329,6001,Alameda County,6,34,25,3.03,25.8,0.3,41.1,0.26,2.76,0,1666753,-2.2,2020-01-04,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,0.0,0,0.9,3.9,5.7,0.3,0.0,0.0
4,67522,California,860.757075,6001,Alameda County,CA,1671329,6001,Alameda County,6,44,29,2.72,24.5,0.4,35.8,0.24,2.48,0,1666753,-12.1,2020-01-05,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,0.0,0.0,0,54.0,0.0,0.0,0.0,0,0,0.0,0,1.3,3.9,5.7,0.3,0.0,0.0


In [37]:
maryland_sd_inertia['State_Name'].unique()

array(['California'], dtype=object)

## Combining both the datasets

In [40]:
print(CA_all_data_selected.shape)
print(maryland_sd_inertia.shape)

print(CA_all_data_selected.Date.min(),CA_all_data_selected.Date.max())

print(maryland_sd_inertia.date.min(),maryland_sd_inertia.date.max())

(15821, 16)
(21054, 51)
2020-03-01 2021-01-05
2020-01-01 2020-12-28


In [41]:
CA_all_datasets = pd.merge(CA_all_data_selected, maryland_sd_inertia, how='inner',left_on=['Date','FIPS'],right_on=['date','countyFIPS'])
print(CA_all_datasets.shape)

CA_all_datasets.head(5)


(15391, 67)


,Date,Province_State,FIPS,New cases,County Name_x,population_x,mask_rule_severity_x,m50,m50_index,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,PopDensity_x,index,State_Name,PopDensity_y,countyFIPS,County Name_y,State,population_y,CTFIPS,CTNAME,STFIPS,Social.distancing.index,X..staying.home,Trips.person,X..out.of.county.trips,X..out.of.state.trips,Miles.person,Work.trips.person,Non.work.trips.person,New.COVID.cases,Population,X..change.in.consumption,date,Transit.mode.share,X..people.older.than.60,Median.income,X..African.Americans,X..Hispanic.Americans,X..Male,Population.density,Employment.density,X..hot.spots.1000.people,Hospital.beds.1000.people,ICUs.1000.people,X..contact.tracing.workers.1000.people,New.cases.1000.people,Active.cases.1000.people,X.days..decreasing.COVID.cases,X..hospital.bed.utilization,Testing.capacity,Tests.done.1000.people,X..ICU.utilization,Ventilator.shortage,Imported.COVID.cases,COVID.exposure.1000.people,X.days..decreasing.ILI.cases,Unemployment.claims.1000.people,Unemployment.rate,X..working.from.home,Cumulative.inflation.rate,COVID.death.rate,mask_rule_severity_y
0,2020-03-01,California,6001,0,Alameda County,1671329,0.0,3.684,58,6.0,9.0,15.0,1.0,4.0,0.0,860.757075,67578,California,860.757075,6001,Alameda County,CA,1671329,6001,Alameda County,6,37,26,2.93,25.4,0.4,36.8,0.25,2.68,0,1666753,-5.1,2020-03-01,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,0.0000,0.0,37,54.0,0.0,0.00,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.00,0.0
1,2020-03-02,California,6001,0,Alameda County,1671329,0.0,6.182,98,4.0,7.0,7.0,-1.0,3.0,0.0,860.757075,67579,California,860.757075,6001,Alameda County,CA,1671329,6001,Alameda County,6,17,20,3.38,28.2,0.4,38.1,0.63,2.74,0,1666753,-2.9,2020-03-02,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,0.0000,0.0,38,54.0,0.0,0.00,0.0,0,0,0.05,35,1.1,5.3,5.7,0.6,0.00,0.0
2,2020-03-03,California,6001,1,Alameda County,1671329,0.0,6.271,100,5.0,12.0,41.0,0.0,1.0,-1.0,860.757075,67580,California,860.757075,6001,Alameda County,CA,1671329,6001,Alameda County,6,16,18,3.61,27.1,0.3,32.7,0.67,2.94,1,1666753,4.1,2020-03-03,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,0.0002,0.0,0,54.0,0.0,0.00,0.0,0,1,0.05,42,1.1,5.3,5.7,0.6,0.00,0.0
3,2020-03-04,California,6001,0,Alameda County,1671329,0.0,6.209,99,1.0,6.0,23.0,-3.0,1.0,0.0,860.757075,67581,California,860.757075,6001,Alameda County,CA,1671329,6001,Alameda County,6,14,18,3.55,27.9,0.3,36.9,0.68,2.87,0,1666753,1.7,2020-03-04,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,0.0002,0.0,0,54.0,10.3,0.01,0.0,0,1,0.06,42,1.1,5.3,5.7,0.6,0.04,0.0
4,2020-03-05,California,6001,0,Alameda County,1671329,0.0,6.654,106,1.0,6.0,12.0,-5.0,-1.0,1.0,860.757075,67582,California,860.757075,6001,Alameda County,CA,1671329,6001,Alameda County,6,15,19,3.54,28.1,0.3,36.8,0.66,2.88,0,1666753,2.0,2020-03-05,15.04,19,92574,10.5,22.5,49.11,2002,1155,131,2.05,0.22,NaN,0.0002,0.0,0,54.0,10.3,0.01,0.0,0,1,0.06,42,1.1,5.3,5.7,0.6,0.04,0.0


In [42]:
print(CA_all_datasets.date.min(),CA_all_datasets.date.max())

2020-03-01 2020-12-28


In [43]:
CA_all_datasets.columns

Index(['Date', 'Province_State', 'FIPS', 'New cases', 'County Name_x',
       'population_x', 'mask_rule_severity_x', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity_x', 'index',
       'State_Name', 'PopDensity_y', 'countyFIPS', 'County Name_y', 'State',
       'population_y', 'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'date', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hisp

In [45]:
required_cols = ['Date', 'Province_State', 'FIPS', 'New cases', 'County Name_x',
       'mask_rule_severity_x', 'm50', 'm50_index',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'PopDensity_x', 
       'State_Name', 'State',
       'CTFIPS', 'CTNAME', 'STFIPS', 'Social.distancing.index',
       'X..staying.home', 'Trips.person', 'X..out.of.county.trips',
       'X..out.of.state.trips', 'Miles.person', 'Work.trips.person',
       'Non.work.trips.person', 'New.COVID.cases', 'Population',
       'X..change.in.consumption', 'Transit.mode.share',
       'X..people.older.than.60', 'Median.income', 'X..African.Americans',
       'X..Hispanic.Americans', 'X..Male',
       'Employment.density', 'X..hot.spots.1000.people',
       'Hospital.beds.1000.people', 'ICUs.1000.people',
       'X..contact.tracing.workers.1000.people', 'New.cases.1000.people',
       'Active.cases.1000.people', 'X.days..decreasing.COVID.cases',
       'X..hospital.bed.utilization', 'Testing.capacity',
       'Tests.done.1000.people', 'X..ICU.utilization', 'Ventilator.shortage',
       'Imported.COVID.cases', 'COVID.exposure.1000.people',
       'X.days..decreasing.ILI.cases', 'Unemployment.claims.1000.people',
       'Unemployment.rate', 'X..working.from.home',
       'Cumulative.inflation.rate', 'COVID.death.rate']

In [46]:
CA_all_datasets_filtered = CA_all_datasets[required_cols]


## Save to csv file 

In [47]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'CA_dataset_'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
CA_all_datasets_filtered.to_csv(location+filename,index=False)

CA_dataset_Feb11.csv
